In [1]:
%load_ext autotime

In [2]:
import pickle

import pandas as pd
from tqdm import tqdm

tqdm.pandas()

with open('./pkl/question_info.pkl', 'rb') as f:
    question_info = pickle.load(f)

with open('./pkl/answer_info.pkl', 'rb') as f:
    answer_info = pickle.load(f)

answer_info['content_w_series'] = answer_info['content_w_series'].progress_apply(
    lambda x: [str(i) for i in x])
question_info['title_w_series'] = question_info['title_w_series'].progress_apply(
    lambda x: [str(i) for i in x])
question_info['desc_w_series'] = question_info['desc_w_series'].progress_apply(
    lambda x: [str(i) for i in x])

100%|██████████| 1829900/1829900 [00:25<00:00, 70954.80it/s] 


time: 3min 22s


In [3]:
question_word_titles = question_info['title_w_series'].values.tolist()
question_word_desc = question_info['desc_w_series'].values.tolist()
answer_word_content = answer_info['content_w_series'].values.tolist()

NameError: name 'Dictionary' is not defined

time: 319 ms


In [4]:
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

dictionary = Dictionary(question_word_titles + question_word_desc + answer_word_content)

time: 6min 16s


In [5]:
tf_idf_model = TfidfModel(dictionary=dictionary)

time: 1.69 s


In [6]:
corpus_q_title = [dictionary.doc2bow(text) for text in question_word_titles]
corpus_q_desc = [dictionary.doc2bow(text) for text in question_word_desc]
corpus_answer = [dictionary.doc2bow(text) for text in answer_word_content]

time: 3min 51s


In [9]:
id2token = {v: k for k, v in dictionary.token2id.items()}

time: 337 ms


In [8]:
tf_idf_model[corpus_q_title[0]]

[(0, 0.6087921374576586), (1, 0.6734316353437473), (2, 0.4193589940468448)]

time: 5.23 ms


In [15]:
def generate_key_words(corpus, top=5):
    key_words_result = []
    for item in tqdm(corpus):
        gensim_dict = {}
        for w, n in tf_idf_model[item]:
            gensim_dict[id2token.get(w)] = n
        key_words = sorted(gensim_dict.items(), key=lambda x: x[1], reverse=True)[:top]
        key_words_result.append([int(k) for k, v in key_words])
    return key_words_result


key_words_q_title_list = generate_key_words(corpus_q_title)

100%|██████████| 1829900/1829900 [02:16<00:00, 13454.21it/s]

time: 2min 16s


In [16]:
key_words_q_desc_list = generate_key_words(corpus_q_desc)
key_words_answer_list = generate_key_words(corpus_answer)

100%|██████████| 4513735/4513735 [15:22<00:00, 4893.55it/s]

time: 18min 2s


In [17]:
print(key_words_q_title_list[0])
print(key_words_q_desc_list[0])
print(key_words_answer_list[0])

[22414, 10458, 963]
[0]
[13100, 20372, 4527, 10528, 10]
time: 1.38 ms


In [18]:
key_words_q_desc_df = pd.DataFrame()
key_words_q_desc_df['key_words_q_desc'] = key_words_q_desc_list
key_words_q_desc_df.to_hdf('./key_words_q_desc_gensim.h5', key='data')

key_words_answer_df = pd.DataFrame()
key_words_answer_df['key_words_answer'] = key_words_answer_list
key_words_answer_df.to_hdf('./key_words_answer_gensim.h5', key='data')

key_words_q_title_df = pd.DataFrame()
key_words_q_title_df['key_words_q_title'] = key_words_q_title_list
key_words_q_title_df.to_hdf('./key_words_q_title_gensim.h5', key='data')

/data/sunrui/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['key_words_q_desc']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
/data/sunrui/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['key_words_answer']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


time: 10.4 s


/data/sunrui/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['key_words_q_title']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [19]:
display(key_words_q_desc_df.head())
display(key_words_answer_df.head())
display(key_words_q_title_df.head())

,key_words_q_desc
0,[0]
1,"[12677, 101839, 16829, 15201, 10609]"
2,"[118371, 194699, 72883, 20788, 219486]"
3,"[12519, 7177, 6642, 3293, 3312]"
4,[0]


,key_words_answer
0,"[13100, 20372, 4527, 10528, 10]"
1,"[53385, 109029, 151399, 66060, 157]"
2,"[107789, 81311, 69108, 31394, 12395]"
3,"[8342, 6485, 9689, 3727, 3700]"
4,"[23420, 8749, 9867, 5506, 1718]"


,key_words_q_title
0,"[22414, 10458, 963]"
1,"[101839, 16829, 15201, 12677, 6419]"
2,"[81215, 3280, 2781, 700]"
3,"[3312, 1823, 1505, 638, 461]"
4,"[2253, 895, 700]"


time: 14.7 ms


In [1]:
import pandas as pd

key_words_q_desc = pd.read_hdf('./key_words_q_desc_gensim.h5', key='data')

In [2]:
key_words_q_desc.head()

,key_words_q_desc
0,[0]
1,"[12677, 101839, 16829, 15201, 10609]"
2,"[118371, 194699, 72883, 20788, 219486]"
3,"[12519, 7177, 6642, 3293, 3312]"
4,[0]


163552